In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tpu-getting-started/sample_submission.csv
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/09-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/14-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/01-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/13-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/00-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/07-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/12-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/11-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/05-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/04-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/val/15-224x224-232.tfrec
/kaggle/input/tpu-getting-started/tfrecord

In [2]:
# Import Library and Framework used.

import tensorflow as tf
print("Tensorflow Version: ", tf.__version__)

from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
import glob

2024-02-29 06:39:03.534409: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 06:39:03.534600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 06:39:03.715718: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Tensorflow Version:  2.15.0


In [3]:
# Detect and connect to TPU if available, otherwise use default strategy for CPU/GPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None 

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [4]:
# Understanding TFRecord files structure

raw_dataset = tf.data.TFRecordDataset("/kaggle/input/tpu-getting-started/tfrecords-jpeg-192x192/val/05-192x192-232.tfrec")

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)
    
# Each records contain several features like class, id, and image.

features {
  feature {
    key: "class"
    value {
      int64_list {
        value: 12
      }
    }
  }
  feature {
    key: "id"
    value {
      bytes_list {
        value: "598ee68f2"
      }
    }
  }
  feature {
    key: "image"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\001\001,\001,\000\000\377\333\000C\000\002\001\001\001\001\001\002\001\001\001\002\002\002\002\002\004\003\002\002\002\002\005\004\004\003\004\006\005\006\006\006\005\006\006\006\007\t\010\006\007\t\007\006\006\010\013\010\t\n\n\n\n\n\006\010\013\014\013\n\014\t\n\n\n\377\333\000C\001\002\002\002\002\002\002\005\003\003\005\n\007\006\007\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\377\300\000\021\010\000\300\000\300\003\001\021\000\002\021\001\003\021\001\377\304\000\036\000\000\001\004\003\001\001\001\000\000\000\000\000\000\000\000\000\007\004\005\006\010\002\003\t\001\000\n\377\304\000H\020\000\001\003\002\0

In [5]:
# Get Google Clooud Storage path
GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started') # it's like /kaggle/input/tpu-getting-started
print(GCS_DS_PATH)

gs://kds-138ac6922774963d086c5172228b1618b34a2e8cbf9e9b8d427510d2


In [6]:
IMAGE_SIZE = [512, 512]
GCS_PATH = GCS_DS_PATH + '/tfrecords-jpeg-512x512'

# AUTOTUNE automatically chooses the optimal parallelism 
# for tf.data operations like map, batch, prefetch based on runtime hardware.
AUTO = tf.data.experimental.AUTOTUNE

# Dataset Path
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

# Flower Classes
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102

In [7]:
# Data Preprocessing Function Definition

def decode_img(img_data):
    image = tf.image.decode_jpeg(img_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0 # cast image to float32 in range [0, 1]
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # reshape image to size 512x512x3, explicit size needed for TPU
    return image
    
def parse_labeled_data(example_proto):
    feature_desc = {
        'class': tf.io.FixedLenFeature([], tf.int64),
        'image': tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example_proto, feature_desc)
    clss = example['class']
    image = decode_img(example['image'])
    return clss, image
    
def parse_unlabeled_data(example_proto):
    feature_desc = {
        'id': tf.io.FixedLenFeature([], tf.string),
        'image': tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example_proto, feature_desc)
    idnum = example['id']
    image = decode_img(example['image'])
    return idnum, image

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.
    ignore_order = tf.data.Options()
    if not ordered:
        # This will make the interleave order non-deterministic, increase speed
        ignore_order.deterministic = False
    
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_calls=AUTO) # num_parallel_calls for parallelism
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(parse_labeled_data if labeled else parse_unlabeled_data, num_parallel_calls=AUTO)
    # returns (class, image) pair if labeled=True or (id, image) pair if labeled=False
    return dataset

In [8]:
"""# Reading TFRecord Data
# Train Dataset
train_list = glob.glob("/kaggle/input/tpu-getting-started/tfrecords-jpeg-192x192/train/*.tfrec")
train_data = tf.data.TFRecordDataset(train_list)
# Validation Dataset
val_list = glob.glob("/kaggle/input/tpu-getting-started/tfrecords-jpeg-192x192/val/*.tfrec")
val_data = tf.data.TFRecordDataset(val_list)
# Test Dataset
test_list = glob.glob("/kaggle/input/tpu-getting-started/tfrecords-jpeg-192x192/test/*.tfrec")
test_data = tf.data.TFRecordDataset(test_list)"""

'# Reading TFRecord Data\n# Train Dataset\ntrain_list = glob.glob("/kaggle/input/tpu-getting-started/tfrecords-jpeg-192x192/train/*.tfrec")\ntrain_data = tf.data.TFRecordDataset(train_list)\n# Validation Dataset\nval_list = glob.glob("/kaggle/input/tpu-getting-started/tfrecords-jpeg-192x192/val/*.tfrec")\nval_data = tf.data.TFRecordDataset(val_list)\n# Test Dataset\ntest_list = glob.glob("/kaggle/input/tpu-getting-started/tfrecords-jpeg-192x192/test/*.tfrec")\ntest_data = tf.data.TFRecordDataset(test_list)'

In [9]:
'''# Parsing Function
# For labeled data like training and validation data
def parse_labeled_data(example_proto):
    feature_desc = {
        'class': tf.io.FixedLenFeature([], tf.int64),
        'image': tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example_proto, feature_desc)
    clss = example['class']
    image = tf.io.decode_jpeg(example['image'], channels=3)
    return clss, image

# For unlabeled data like test data
def parse_unlabeled_data(example_proto):
    feature_desc = {
        'id': tf.io.FixedLenFeature([], tf.string),
        'image': tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example_proto, feature_desc)
    idnum = example['id']
    image = tf.io.decode_jpeg(example['image'], channels=3)
    return idnum, image

# Apply the Parse Function into Dataset
train_data = train_data.map(parse_labeled_data)
val_data = val_data.map(parse_labeled_data)
test_data = test_data.map(parse_unlabeled_data)'''

"# Parsing Function\n# For labeled data like training and validation data\ndef parse_labeled_data(example_proto):\n    feature_desc = {\n        'class': tf.io.FixedLenFeature([], tf.int64),\n        'image': tf.io.FixedLenFeature([], tf.string)\n    }\n    example = tf.io.parse_single_example(example_proto, feature_desc)\n    clss = example['class']\n    image = tf.io.decode_jpeg(example['image'], channels=3)\n    return clss, image\n\n# For unlabeled data like test data\ndef parse_unlabeled_data(example_proto):\n    feature_desc = {\n        'id': tf.io.FixedLenFeature([], tf.string),\n        'image': tf.io.FixedLenFeature([], tf.string)\n    }\n    example = tf.io.parse_single_example(example_proto, feature_desc)\n    idnum = example['id']\n    image = tf.io.decode_jpeg(example['image'], channels=3)\n    return idnum, image\n\n# Apply the Parse Function into Dataset\ntrain_data = train_data.map(parse_labeled_data)\nval_data = val_data.map(parse_labeled_data)\ntest_data = test_data.

In [10]:
"""for element in train_data.take(1):
    print(element)"""

'for element in train_data.take(1):\n    print(element)'

In [11]:
"""for clss, _id, img in train_data.take(2):
    plt.figure()
    plt.imshow(img.numpy())
    plt.title("Class: " + str(clss.numpy()))
    plt.axis('off')

plt.show()"""

'for clss, _id, img in train_data.take(2):\n    plt.figure()\n    plt.imshow(img.numpy())\n    plt.title("Class: " + str(clss.numpy()))\n    plt.axis(\'off\')\n\nplt.show()'